## Members

We now have destructors implemented for all our classes, and they are called
automatically and implicitly when we exit functions.

The next thing to discuss is composability. After all, we want to keep
our objects as members for any actual use.


Well, that should be fairly simple!
We just take our value and store it in a member variable.

In [ ]:
class BetterArchiveReader:
    zipfile: ZipFile

    def BetterArchiveReader(self, path: str):
        self.zipfile = ZipFile(path)

    def read(self, name: str):
        with self.zipfile.open(name) as f:
            return f.read()

    def _BetterArchiveReader(self):
        self.zipfile.close()

In [ ]:
class BetterArchiveReader:
    zipfile: ZipFile

    def BetterArchiveReader(self, path: str):
        self.zipfile = ZipFile(path)
        remove_dtor(self.zipfile)

    def read(self, name: str):
        with self.zipfile.open(name) as f:
            return f.read()

    def _BetterArchiveReader(self):
        self.zipfile.close()

In [ ]:
class Reader:
    def __init__(self, path):
        self.zipfile = ZipFile(path)


Well, we _do_ have a member now, but we also have a dangling reference.
As soon as we leave the constructor, the destructor for `ZipFile` will be called,
and our member will be destructed. Any usage will result in undefined behaviour.

For members to actually work, we need to remove them from the destuctor-scope
of the current function.

To do this, we need to go back to our `DtorScope` and add another method:

In [ ]:
class DtorScope:
    stack: list
    ...
    def remove(self, cm):
        self.stack.remove(cm)

This won't work, though. `deque.remove()` removes by equality, not identity.
To remedy this, we'll borrow a trick from the C++ playbook and use a
comparison function object.

In [ ]:
class IdentityComparator:
    def __init__(self, obj):
        self.obj = obj

    def __eq__(self, other):
        return self.obj is other


class DtorScope:
    ...
    def remove(self, cm):
        self.stack.remove(IdentityComparator(cm))

def remove_dtor(cm):
    get_dtor_stack()[-1].remove(cm)

By changing the equality operator for our wrapper to check for identity,
we can easily remove our object from the `DtorScope`.

In [ ]:
class Reader:
    def Reader(self, path):
        self.zipfile = ZipFile(path)
        remove_dtor(self.zipfile)

Now we know that it isn't destructed at the end of the scope,
but we still need to destruct it somewhere.

In [ ]:
class Reader:
    ...
    def _Reader(self):
        self.zipfile.__exit__(None, None, None)

Not pretty, but it works.


But now we're back to manually trigerring our dtor mechanisms, and no one
wants that.

Instead, we want the removal from the scope to happen automatically on
assignment, and cleanup to happen automatically in our class' dtor.

Unlike C++, Python does not have an assignment operator.
It does, however, allow for getters and setters for member variables.

Before we go into the actual Python mechanism, let's write our getter
and setter using naive Python code.

In [ ]:
class Reader:
    def get_zipfile(self):
        return self.zipfile

    def set_zipfile(self, zipfile):
        old = getattr(self, "zipfile", None)

        if isinstance(old, CppClass):
            old.__exit__(None, None, None)

        if isinstance(zipfile, CppClass):
            remove_dtor(zipfile)

        self.zipfile = zipfile

    def Reader(self, path):
        self.set_zipfile(ZipFile(path))

Our getter `get_zipfile` is as straightforward as can be.
We just return the value, as we won't allow moving out of members.

Our setter, on the other hand, has a bit of work to do.
If an existing file exists, it triggers it's dtor.
Then, it removes the new value from the scope, and sets the value.

Note that this is pseudo code - now that we auto-decorate all member
functions, `remove_dtor` will remove from the wrong scope!

But, now that we know what our getter and setter need to do,
we can go on to implement them in a cleaner fashion.

In Python, if a class member variable implements `__get__` and `__set__`
methods, they will be called on instance member access to provide
getters and setters.
Such objects are called Descriptors.

Just like our plain function-based getters, the new object will also
a member variable of the hosting class to store the value.

In [ ]:
class CppMember:
    def __set_name__(self, owner, name):
        self.private_name = "_" + name

    def __get__(self, instance, owner=None):
        return getattr(instance, self.private_name)

    def __set__(self, instance, value):
        old = getattr(instance, self.private_name, None)

        if isinstance(old, CppClass):
            old.__exit__(None, None, None)

        if isinstance(value, CppClass):
            remove_dtor(value)

        setattr(instance, self.private_name, value)

class Reader:
    zipfile = CppMember()
    def Reader(self, path):
        self.zipfile = ZipFile(path)


        self.zipfile.__get__(self)
        zipfile.__set__(self, ZipFile(path))
        BetterArchiveReader.zipfile.__set_name__(BetterArchiveReader, "zipfile")

The `__set_name__` will be automatically called by Python whe the
class (not the instance!) is created, to pass the variable name
into the descriptor.

Since our descriptor will occupy the user's choice of a name, we prefix
the name so that we can use it for our own variable storage.

With that we can freely assign member variables and avoid use-after-free.
But we still need to free them at some point...

To fix that, we'll have to add some code to the destructor.
We'll go through all the members, and if they're a `CppClass` object,
we'll destruct them.

In [ ]:
def cpp_class(cls):
    ...
    def __exit__(self, exc_type, exc_val, exc_tb):
        dtor = getattr(self, "_"+self.__class__.__name__, None)
        if dtor:
            dtor()

        with DtorScope():
            for _, value in inspect.getmembers(self):
                push_dtor(value)

All we have to do is push our members into a new DtorScope, and
they'll get automatically destructed.

**Need to add code sample here, showing our class with members!**

In [ ]:
class Reader:
    zipfile = CppMember()
    def Reader(self, path):
        self.zipfile = ZipFile(path)


And we're good to go. But we can go a step further, and hide
the instantiation of `CppMember()`.
Python now allows type-annotation. They don't do anything, but they
can be queried.
So instead of assigning a `CppMember` for every member variable,
we'll just annotate them with the type they are intended to hold,
and let our type-constructor do the rest.
We can access `dct['__annotations__']` for a mapping off all the annotations
in a class.

In [ ]:
def create_members(cls):
    member_names = list(getattr(cls, '__annotations__', {}))

    for name in member_names:
        member = CppMember()
        member.__set_name__(cls, name)
        setattr(cls, name, member)

    setattr(cls, '__member_names__', member_names)

def cpp_class(cls):
    ...
    create_members(cls)
    ...


class Reader:
    zipfile: ZipFile
    def Reader(self, path):
        self.zipfile = ZipFile(path)

**TODO: add the complete code sample here!

And with that, we've achieved our initial goal.